### Неделя 7: Интерактивное демо 

In [7]:
import os
import numpy as np
import pandas as pd
import json
import folium
import matplotlib.pyplot as plt
import base64
from folium import IFrame
from ipywidgets import interact

In [8]:
# Можете исполнить, чтобы сделать отображение ноутбука пошире. Исключительно необязательно для исполнения.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [9]:
# Индексы регионов (102 шт.) отобранных на 2 неделе
reg_ind = pd.read_csv('ids.csv',index_col=0)
reg_ind = reg_ind.index.values

In [10]:
# Грузим исходные данные и предсказания
df = pd.read_csv('original_data.csv',parse_dates=['time',],index_col='time')
df.columns=reg_ind
dft = df.T
dft['id'] = [str(ind) for ind in reg_ind]

df_arima = pd.read_csv('arima_data.csv',parse_dates=['time',],index_col='time')
df_arima.columns=reg_ind
df_arimat = df_arima.T
df_arimat['id'] = [str(ind) for ind in reg_ind]

In [11]:
# Берем geojson координаты регионов для интерактивной карты
import json
with open('filtered_cells.json','rb') as f:
    handle = f.read()
fc = json.loads(handle)

### Рисуем новую  интерактивную карту

1) На каждую ячейку можно нажать, чтобы увидеть информацию о регионе (предсказанный и реально зафиксированный спрос) в данной временной точке, а также графики временных рядов с настраиваемым масштабом 

2) Ползунок позволяет просматривать разные временные точки

3) Возможен выбор разных цветовых схем отображения, а также переключение между визуализацией реального и предсказанного спроса

In [12]:

@interact(color_scheme = ['BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu',
    'YlGn', 'YlGnBu', 'YlOrBr','YlOrRd'],data_type=['acutal','predicted'],time_point = (0,9503),
          plot_range = (50,1000),plot_region = reg_ind)


def chloromap(color_scheme, data_type,time_point,plot_range,plot_region):
    print 'Time: {}'.format(dft.columns[time_point])
    
    m1 = folium.Map(control_scale=True,location=[40.6892494,-73.9045004],tiles='CartoDB positron',zoom_start=11)
    
    
  
    
    if data_type == 'predicted':
        
        m1.choropleth(highlight=False,
            geo_data='filtered_cells.json',
            name='choropleth',
            data=df_arimat,
            columns=['id', df_arimat.columns[time_point]],
            key_on='feature.id',
            fill_color=color_scheme,
            fill_opacity=0.7,
            line_opacity=0.3,
            legend_name='Taxi rides'
        )
    
    else:
        m1.choropleth(highlight=False,
            geo_data='filtered_cells.json',
            name='choropleth',
            data=dft,
            columns=['id', dft.columns[time_point]],
            key_on='feature.id',
            fill_color=color_scheme,
            fill_opacity=1,
            line_opacity=0.3,
            legend_name='Taxi rides'
            
        )
        
    style_function = lambda feature : dict(
    color='g',
    weight=0.2,
    opacity=0)
    
    
    
    for i in xrange(len(fc['features'])):
        reg_id = fc['features'][i]['id']
        
        html = """        <h2> Region {}</h2><br>
        Time: {}<br>
        Predicted demand:
        {}<br>
        Actual demand:
        {}<br>""".format(reg_id,dft.columns[time_point],df_arimat.iloc[(list(reg_ind)).index(int(reg_id)),time_point],
                         dft.iloc[(list(reg_ind)).index(int(reg_id)),time_point])
        gj = folium.GeoJson(
        data=fc['features'][i]['geometry'],style_function= style_function)
        gj.add_child(folium.Popup(html) )
        gj.add_to(m1)
    folium.LayerControl().add_to(m1)
    
    left = - plot_range + time_point
    right = time_point + plot_range
    if left<0: left = 0
    if right> df.shape[0]-1: right = df.shape[0]-1

    
    fig = plt.figure(figsize=(20,10))
    plt.subplot(2, 2, 1)
    plt.ylabel('Taxi demand time series',fontsize=24)
    
    df_arima[plot_region].iloc[left:right].plot(color = 'r',label = 'Predicted')
    plt.legend()
    plt.subplot(2, 2, 2)
    df_arima[plot_region].iloc[left:right].plot(label='Actual')
    plt.legend()
    plt.tight_layout()
    
    plt.show()

    
    

       
    return m1
    


SyntaxError: invalid syntax (<ipython-input-12-848c211a71c8>, line 8)